In [1]:
import pandas as pd
import numpy as np

In [163]:
def isNaN(num):
    return num != num

In [43]:
cell_anot = pd.read_csv('../data/small_screen/cell.annotations.txt', sep='\t', header=None, names=['cell_annotation', 'f1'])

In [44]:
cell_anot

,cell_annotation,f1
0,A01_A01_RT_384,sciChem18
1,A01_A01_RT_389,sciChem18
2,A01_A01_RT_390,sciChem18
3,A01_A01_RT_396,sciChem18
4,A01_A01_RT_397,sciChem18
...,...,...
24257,H12_B02_RT_752,sciChem18
24258,H12_B02_RT_753,sciChem18
24259,H12_B02_RT_760,sciChem18
24260,H12_B02_RT_761,sciChem18


In [37]:
hash_table = pd.read_csv('../data/small_screen/hashTable.out.txt', sep='\t', header=None, names=['f1', 'cell_annotation', 'general', 'f2', 'f3'])

In [38]:
hash_table

,f1,cell_annotation,general,f2,f3
0,sciChem18,A01_A01_RT_1424,Dex_0_BC08,1,1
1,sciChem18,A01_A01_RT_1424,Nutlin_0.5_BB02,1,1
2,sciChem18,A01_A01_RT_1424,Nutlin_50_BA10,1,1
3,sciChem18,A01_A01_RT_1424,SAHA_0_BF12,1,2
4,sciChem18,A01_A01_RT_15,Dex_100_AC09,1,1
...,...,...,...,...,...
350176,sciChem18,H12_B02_RT_767,Dex_1_AD09,1,1
350177,sciChem18,H12_B02_RT_829,Nutlin_0_BB04,1,1
350178,sciChem18,H12_B02_RT_945,Dex_100_BC09,1,1
350179,sciChem18,H12_B02_RT_997,BMS_0.5_AG06,1,1


In [177]:
hash_table.loc[hash_table['cell_annotation'] == 'A01_A01_RT_396', ]

,f1,cell_annotation,general,f2,f3
23,sciChem18,A01_A01_RT_396,BMS_0.1_BG07,1,8
24,sciChem18,A01_A01_RT_396,BMS_0_BH04,1,1
25,sciChem18,A01_A01_RT_396,BMS_100_BG09,1,1
26,sciChem18,A01_A01_RT_396,BMS_5_AH08,1,1
27,sciChem18,A01_A01_RT_396,Dex_100_AD05,1,1
28,sciChem18,A01_A01_RT_396,Dex_50_BC10,1,19
29,sciChem18,A01_A01_RT_396,Nutlin_0_AB04,1,1
30,sciChem18,A01_A01_RT_396,Nutlin_10_AA03,1,1
31,sciChem18,A01_A01_RT_396,SAHA_100_AE01,1,1
32,sciChem18,A01_A01_RT_396,SAHA_5_AF08,1,1


In [36]:
hash_sample = pd.read_csv('../data/small_screen/hashSampleSheet.txt', sep='\t', header=None, names=['general', 'barcodes', 'f0'], usecols=[0, 1, 2])
hash_sample

,general,barcodes,f0
0,Nutlin_100_AA01,CGGTCAAGAA,1
1,Nutlin_50_AA02,CGCTCCTAAC,1
2,Nutlin_10_AA03,ATCCATGACT,1
3,Nutlin_5_AA04,AACCTGGTCT,1
4,Nutlin_1_AA05,ACCGAAGACC,1
...,...,...,...
187,BMS_5_BH08,GCGAGCGCTC,1
188,BMS_1_BH09,TCGCAAGACG,1
189,BMS_0.5_BH10,CAATCTCGGA,1
190,BMS_0.1_BH11,TCGACCTACC,1


In [50]:
cell_hash_table = pd.merge(left=cell_anot, right=hash_table, on='cell_annotation', how='left').drop_duplicates(subset='cell_annotation')

In [164]:
everything = pd.merge(left=cell_hash_table, right=hash_sample, on='general', how='left')

In [165]:
everything.drop(columns=['f1_x', 'f1_y', 'f2', 'f3', 'f0'], inplace=True)

In [166]:
everything.iloc[215, ]

cell_annotation    A01_B02_RT_742
general                       NaN
barcodes                      NaN
Name: 215, dtype: object

In [167]:
separate_cols = pd.DataFrame(everything['general'].str.split('_').tolist(), 
                       columns = ['tr|eatment', 'dose', 'cell'])
separate_cols

TypeError: object of type 'float' has no len()

In [168]:
everything.loc[isNaN(everything['general']), 'general'] = 'NoinfoT_NoinfoD_NoinfoC'
everything.loc[isNaN(everything['barcodes']), 'barcodes'] = 'NoinfoB'

In [172]:
separate_cols = pd.DataFrame(everything['general'].str.split('_').tolist(), 
                       columns = ['treatment', 'dose', 'cell'])
separate_cols

,treatment,dose,cell
0,BMS,0.1,AG07
1,BMS,0,AG08
2,BMS,0.1,BH11
3,BMS,0.1,BG07
4,BMS,0.1,BH03
...,...,...,...
24257,BMS,0.1,BG07
24258,BMS,0.1,BH11
24259,BMS,0.5,AG06
24260,BMS,0,AG08


In [174]:
final_data = pd.concat((separate_cols, everything[['cell_annotation', 'barcodes']]), axis=1)
final_data = final_data[['cell_annotation', 'cell', 'treatment', 'dose', 'barcodes']]
final_data

,cell_annotation,cell,treatment,dose,barcodes
0,A01_A01_RT_384,AG07,BMS,0.1,TTCTTCCGGT
1,A01_A01_RT_389,AG08,BMS,0,AGGAGAGAAC
2,A01_A01_RT_390,BH11,BMS,0.1,TCGACCTACC
3,A01_A01_RT_396,BG07,BMS,0.1,ACCGGCCGAC
4,A01_A01_RT_397,BH03,BMS,0.1,CGGTCTCATA
...,...,...,...,...,...
24257,H12_B02_RT_752,BG07,BMS,0.1,ACCGGCCGAC
24258,H12_B02_RT_753,BH11,BMS,0.1,TCGACCTACC
24259,H12_B02_RT_760,AG06,BMS,0.5,CCGCTCCGGC
24260,H12_B02_RT_761,AG08,BMS,0,AGGAGAGAAC
